In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [6]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [7]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [8]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [9]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [10]:
def delete_age_time(df, flag):

    days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']

    age = '연령대'
    del_columns = df.filter(like=age).columns
    df.drop(del_columns, axis=1, inplace=True)
    
    times = '시간대'    
    del_columns = df.filter(like=times).columns
    df.drop(del_columns, axis=1, inplace=True)
    
    tmp_arr = []
    for day in days[5:]:
        day = flag + day
        tmp_arr.append(day+'_유동인구_수')
    df[flag+'주말_유동인구수'] = df[tmp_arr].sum(axis=1)
    df.drop(tmp_arr, axis=1, inplace=True)
    
    tmp_arr = []
    for day in days[:-2]:
        day = flag + day
        tmp_arr.append(day+'_유동인구_수')
    df[flag+'주중_유동인구수'] = df[tmp_arr].sum(axis=1)
    df.drop(tmp_arr, axis=1, inplace=True)
    
    return df    

In [11]:
def cat_df(sg, sg_hl):
    df = load_to_df(sg)
    df0 = df.copy()
    df0["상권/배후지"] = 0
    
    df = load_to_df(sg_hl)
    df_hl = df.copy()
    df_hl["상권/배후지"] = 1
    
    new_columns = list(map( lambda s: s.replace('hl_',''), df_hl.columns ))
    df_hl.columns = new_columns
    
    tmp_df = pd.concat([df0,df_hl])
    return tmp_df.loc[:, (tmp_df != 0).any(axis=0)] 

In [12]:
def pred_selng(df):
    del_columns = df.filter(like='시간대_').filter(like='_매출')
    df.drop(del_columns, axis=1, inplace=True)
    del_columns = df.filter(like='연령대_').filter(like='_매출')
    df.drop(del_columns, axis=1, inplace=True)
    del_columns = df.filter(like='요일_')
    df.drop(del_columns, axis=1, inplace=True)
    
    tmp_dt = {'주중_건당_매출' : ['주중_매출_금액', '주중_매출_건수'],
              '주말_건당_매출' : ['주말_매출_금액', '주말_매출_건수'],
              '남성_건당_매출' : ['남성_매출_금액', '남성_매출_건수'],
              '여성_건당_매출' : ['여성_매출_금액', '여성_매출_건수']}

    for key, value in tmp_dt.items():
        join_grouped[key]=join_grouped[value[0]]/join_grouped[value[1]]
    
    return join_grouped

# 다이나믹 기준 보기

In [13]:
ch_df = load_to_df('상권-상권변화지표')
ch_df['평균_폐업_비율'] = ch_df['폐업_영업_개월_평균']/ch_df['서울_폐업_영업_개월_평균']
ch_df['평균_운영_비율'] = ch_df['운영_영업_개월_평균']/ch_df['서울_운영_영업_개월_평균']

In [14]:
ch_df

,기준_년_분기_코드,상권_코드,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,평균_폐업_비율,평균_운영_비율
0,2020_1,1001496,HH,정체,118,61,110,54,1.129630,1.072727
1,2020_1,1001495,LL,다이나믹,103,53,110,54,0.981481,0.936364
2,2020_1,1001494,HH,정체,147,71,110,54,1.314815,1.336364
3,2020_1,1001493,HH,정체,119,61,110,54,1.129630,1.081818
4,2020_1,1001492,HH,정체,147,62,110,54,1.148148,1.336364
...,...,...,...,...,...,...,...,...,...,...
37395,2014_4,1001258,LL,다이나믹,67,45,94,46,0.978261,0.712766
37396,2014_4,1001257,LL,다이나믹,83,45,94,46,0.978261,0.882979
37397,2014_4,1001256,LL,다이나믹,91,43,94,46,0.934783,0.968085
37398,2014_4,1001255,LL,다이나믹,65,38,94,46,0.826087,0.691489


In [15]:
ch_df[(ch_df['기준_년_분기_코드']=='2019_1')
      &(ch_df['상권_변화_지표_명'] == '상권확장')][['상권_코드', '상권_변화_지표', '상권_변화_지표_명', '평균_운영_비율', '평균_폐업_비율']]

,상권_코드,상권_변화_지표,상권_변화_지표_명,평균_운영_비율,평균_폐업_비율
5607,1001110,LH,상권확장,0.887850,1.018868
5613,1001104,LH,상권확장,0.971963,1.113208
5641,1001076,LH,상권확장,0.841121,1.056604
5660,1001057,LH,상권확장,0.915888,1.000000
5666,1001051,LH,상권확장,0.906542,1.037736
...,...,...,...,...,...
7519,1000268,LH,상권확장,0.850467,1.075472
7551,1000236,LH,상권확장,0.813084,1.018868
7560,1000227,LH,상권확장,0.990654,1.075472
7561,1000226,LH,상권확장,0.887850,1.094340


In [16]:
pd.get_dummies(ch_df['상권_변화_지표'])

,HH,HL,LH,LL
0,1,0,0,0
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
37395,0,0,0,1
37396,0,0,0,1
37397,0,0,0,1
37398,0,0,0,1


In [17]:
def unique_sg_cd(sg, sg_hl, yy_qd):
    a_df = load_to_df(sg)
    b_df = load_to_df(sg_hl)
    
    a_df = a_df[a_df['기준_년_분기_코드']== yy_qd]
    b_df = b_df[b_df['기준_년_분기_코드']== yy_qd]
    
    len(set(a_df['상권_코드'].unique())-set(b_df['상권_코드'].unique()))
    a_count = len(a_df['상권_코드'].unique())
    b_count = len(b_df['상권_코드'].unique())
    count = len(a_df['상권_코드'].unique()) - len(set(a_df['상권_코드'].unique())-set(b_df['상권_코드'].unique()))
    del a_df, b_df
    print(yy_qd, a_count, b_count, count, sep=' : ')

In [18]:
# 2019_1

In [19]:
# pop
unique_sg_cd('상권-추정유동인구', '상권배후지-추정유동인구', '2019_1')
unique_sg_cd('상권-상주인구', '상권배후지-상주인구', '2019_1')
unique_sg_cd('상권-직장인구', '상권배후지-직장인구', '2019_1')

# selling
unique_sg_cd('상권-추정매출', '상권배후지-추정매출', '2019_1')

# place
unique_sg_cd('상권-집객시설', '상권배후지-집객시설', '2019_1')
unique_sg_cd('상권-아파트', '상권배후지-아파트', '2019_1')
unique_sg_cd('상권-점포', '상권배후지-점포', '2019_1')


2019_1 : 1495 : 1010 : 1010
2019_1 : 1481 : 1010 : 1010
2019_1 : 1492 : 1010 : 1010
2019_1 : 1488 : 1010 : 1010
2019_1 : 1462 : 1010 : 1008
2019_1 : 1322 : 1009 : 990
2019_1 : 1496 : 1010 : 1010


In [20]:
# 각 배후지의 상권_코드 일치성 확인

In [21]:
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-상주인구', '2019_1')
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-직장인구', '2019_1')
# 추정매출은 분기별로 상권과 배후지의 기복이 심하다.
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-추정매출', '2019_1')

unique_sg_cd('상권배후지-추정유동인구', '상권배후지-집객시설', '2019_1')

unique_sg_cd('상권배후지-추정유동인구', '상권배후지-아파트', '2019_1')
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-점포', '2019_1')

2019_1 : 1010 : 1010 : 1010
2019_1 : 1010 : 1010 : 1010
2019_1 : 1010 : 1010 : 1010
2019_1 : 1010 : 1010 : 1010
2019_1 : 1010 : 1009 : 1009
2019_1 : 1010 : 1010 : 1010


In [22]:
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-소득소비', '2019_1')

2019_1 : 1010 : 1010 : 1010


In [23]:
# 각 상권의 상권_코드 일치성 확인

In [24]:
unique_sg_cd('상권-추정유동인구', '상권-상주인구', '2019_1')
unique_sg_cd('상권-추정유동인구', '상권-직장인구', '2019_1')
# 추정매출은 분기별로 상권과 배후지의 기복이 심하다.

unique_sg_cd('상권-추정유동인구', '상권-점포', '2019_1')

unique_sg_cd('상권-추정유동인구', '상권-추정매출', '2019_1')
unique_sg_cd('상권-추정유동인구', '상권-집객시설', '2019_1')

unique_sg_cd('상권-추정유동인구', '상권-아파트', '2019_1')

2019_1 : 1495 : 1481 : 1481
2019_1 : 1495 : 1492 : 1491
2019_1 : 1495 : 1496 : 1495
2019_1 : 1495 : 1488 : 1488
2019_1 : 1495 : 1462 : 1461
2019_1 : 1495 : 1322 : 1322


In [25]:
# 2019_2

In [26]:
# pop
unique_sg_cd('상권-추정유동인구', '상권배후지-추정유동인구', '2019_2')
unique_sg_cd('상권-상주인구', '상권배후지-상주인구', '2019_2')
unique_sg_cd('상권-직장인구', '상권배후지-직장인구', '2019_2')

# selling
unique_sg_cd('상권-추정매출', '상권배후지-추정매출', '2019_2')

# place
unique_sg_cd('상권-집객시설', '상권배후지-집객시설', '2019_2')
unique_sg_cd('상권-아파트', '상권배후지-아파트', '2019_2')
unique_sg_cd('상권-점포', '상권배후지-점포', '2019_2')


2019_2 : 1495 : 1010 : 1010
2019_2 : 1481 : 1010 : 1010
2019_2 : 1493 : 1010 : 1010
2019_2 : 1486 : 628 : 628
2019_2 : 1462 : 1010 : 1008
2019_2 : 1322 : 1009 : 990
2019_2 : 1496 : 1010 : 1010


In [27]:
# 각 배후지의 상권_코드 일치성 확인

In [28]:
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-상주인구', '2019_2')
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-직장인구', '2019_2')
# 추정매출은 분기별로 상권과 배후지의 기복이 심하다.
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-추정매출', '2019_2')

unique_sg_cd('상권배후지-추정유동인구', '상권배후지-집객시설', '2019_2')

unique_sg_cd('상권배후지-추정유동인구', '상권배후지-아파트', '2019_2')
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-점포', '2019_2')

2019_2 : 1010 : 1010 : 1010
2019_2 : 1010 : 1010 : 1010
2019_2 : 1010 : 628 : 628
2019_2 : 1010 : 1010 : 1010
2019_2 : 1010 : 1009 : 1009
2019_2 : 1010 : 1010 : 1010


In [29]:
unique_sg_cd('상권배후지-추정유동인구', '상권배후지-소득소비', '2019_2')

2019_2 : 1010 : 1010 : 1010


In [30]:
# 각 상권의 상권_코드 일치성 확인

In [31]:
unique_sg_cd('상권-추정유동인구', '상권-상주인구', '2019_2')
unique_sg_cd('상권-추정유동인구', '상권-직장인구', '2019_2')
# 추정매출은 분기별로 상권과 배후지의 기복이 심하다.

unique_sg_cd('상권-추정유동인구', '상권-점포', '2019_2')

unique_sg_cd('상권-추정유동인구', '상권-추정매출', '2019_2')
unique_sg_cd('상권-추정유동인구', '상권-집객시설', '2019_2')

unique_sg_cd('상권-추정유동인구', '상권-아파트', '2019_2')

2019_2 : 1495 : 1481 : 1481
2019_2 : 1495 : 1493 : 1492
2019_2 : 1495 : 1496 : 1495
2019_2 : 1495 : 1486 : 1486
2019_2 : 1495 : 1462 : 1461
2019_2 : 1495 : 1322 : 1322


In [32]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del a_df, b_df
#     all_df.reset_index(['상권_코드','기준_년_분기_코드'])
    if('집객시설' in sg) or ('아파트' in sg):
        all_df.fillna(0, inplace=True)
        return all_df
    
    return all_df.dropna(axis=0)

In [33]:
def sg_OneHotEncode(dest_df):
    # 상권 변화 지표 합치기
    sg_df = load_to_df('상권-상권변화지표')
    sg_df = sg_df[(sg_df['기준_년_분기_코드']=='2019_1')|(sg_df['기준_년_분기_코드']=='2019_2')]
    target_df = pd.merge(dest_df, sg_df[['기준_년_분기_코드', '상권_코드', '상권_변화_지표']],on=['기준_년_분기_코드', '상권_코드'])
#     # 상권 지표 OneHotEncode 변경
#     one_hot_encoding = pd.get_dummies(target_df['상권_변화_지표'])
#     target_df = pd.concat([pd.DataFrame(target_df), one_hot_encoding],axis=1)

#     return target_df.drop('상권_변화_지표',axis=1)
    return target_df


In [34]:
#### 상권/배후지 집객시설 (02, 04)
jip_df = multi_columns_merging('상권-집객시설', '상권배후지-집객시설')
jip_df

,기준_년_분기_코드,상권_코드,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,...,hl_대학교_수,hl_백화점_수,hl_슈퍼마켓_수,hl_극장_수,hl_숙박_시설_수,hl_공항_수,hl_철도_역_수,hl_버스_터미널_수,hl_지하철_역_수,hl_버스_정거장_수
0,2019_2,1000953,14,0,0,0,0,3,0,0,...,0.0,0.0,0.0,11.0,13.0,0.0,0.0,0.0,2.0,30.0
1,2019_2,1001361,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019_2,1001387,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019_2,1001366,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019_2,1001308,4,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,2019_1,1000982,12,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
2920,2019_1,1001369,3,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2921,2019_1,1000404,15,1,0,0,1,2,1,1,...,0.0,0.0,1.0,0.0,11.0,0.0,0.0,0.0,1.0,30.0
2922,2019_1,1000080,7,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,6.0


In [35]:
len(jip_df['상권_코드'].unique())

1462

In [36]:
#### 상권/배후지 아파트 (07, 16)
apart_df = multi_columns_merging('상권-아파트', '상권배후지-아파트')
columns_1 = apart_df.filter(like='아파트_면적_').columns
columns_2 = apart_df.filter(like='아파트_가격_').columns
apart_df.drop(columns_1, axis=1, inplace=True)
apart_df.drop(columns_2, axis=1, inplace=True)

In [37]:
len(apart_df['상권_코드'].unique())

1322

In [38]:
apart_df

,기준_년_분기_코드,상권_코드,아파트_단지_수,아파트_평균_면적,아파트_평균_시가,hl_아파트_단지_수,hl_아파트_평균_면적,hl_아파트_평균_시가
0,2019_2,1001495,20,88,431622104,0.0,0.0,0.0
1,2019_1,1001495,20,88,431622104,0.0,0.0,0.0
2,2019_2,1001494,8,39,103667956,0.0,0.0,0.0
3,2019_1,1001494,8,39,103667956,0.0,0.0,0.0
4,2019_2,1001493,6,69,144042180,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2639,2019_1,1000134,43,50,151913640,213.0,52.0,148594250.0
2640,2019_2,1000133,57,51,150264017,431.0,55.0,154300553.0
2641,2019_1,1000133,57,51,150264017,431.0,55.0,154300553.0
2642,2019_2,1000132,76,53,142053286,292.0,54.0,164267098.0


In [39]:
def sg_svc_category_merge(df):
    grouped = df.groupby(['기준_년_분기_코드','상권_코드'])
    grouped_part1 = grouped.count() # 각 상권별 서비스 업종 코드수    
    grouped_part2 = grouped.sum() # 각 상권별 서비스 업종 분류수    
    join_grouped = grouped_part1.iloc[:,:1].join(grouped_part2, on=['기준_년_분기_코드', '상권_코드'], how='inner')
    return join_grouped

In [40]:
def multi_columns_merging_svc_cd(sg, sg_hl):
    sg_df = load_to_df(sg)
    hl_sg_df = load_to_df(sg_hl)
    
    sg_df = sg_svc_category_merge(sg_df)
    hl_sg_df = sg_svc_category_merge(hl_sg_df)
    
    hl_sg_df.rename(columns={'서비스_업종_코드':'hl_서비스_업종_코드'})
    
    all_df = pd.merge(left=sg_df, right=hl_sg_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
    del sg_df, hl_sg_df
    
# jumpo_df.index.name
# 멀티 인덱스 제거
    
    return all_df.dropna(axis=0).reset_index(['상권_코드','기준_년_분기_코드'])

In [41]:
jumpo_df = multi_columns_merging_svc_cd('상권-점포', '상권배후지-점포')
jumpo_df.rename(columns={'서비스_업종_코드_x':'서비스_업종_코드', '서비스_업종_코드_y':'hl_서비스_업종_코드'}, inplace=True)
jumpo_df

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,hl_서비스_업종_코드,hl_점포_수,hl_유사_업종_점포_수,hl_개업_율,hl_개업_점포_수,hl_폐업_률,hl_폐업_점포_수,hl_프랜차이즈_점포_수
0,2019_1,1000001,71,233,244,44,4,41,4,11,84.0,687.0,741.0,36.0,13.0,67.0,15.0,54.0
1,2019_1,1000002,80,142,149,211,3,134,11,7,99.0,1372.0,1467.0,202.0,39.0,246.0,61.0,95.0
2,2019_1,1000003,81,322,336,211,9,189,8,14,97.0,3142.0,3268.0,182.0,73.0,297.0,86.0,126.0
3,2019_1,1000004,56,97,109,14,2,31,3,12,77.0,258.0,290.0,72.0,7.0,75.0,10.0,32.0
4,2019_1,1000005,58,77,81,33,1,213,5,4,74.0,190.0,198.0,190.0,7.0,108.0,9.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1001006,77,144,164,150,9,102,6,20,89.0,753.0,825.0,285.0,26.0,508.0,31.0,72.0
2016,2019_2,1001007,78,169,181,195,8,265,9,12,87.0,476.0,516.0,202.0,17.0,142.0,16.0,40.0
2017,2019_2,1001008,80,206,220,109,10,35,5,14,97.0,1764.0,1905.0,266.0,45.0,245.0,57.0,141.0
2018,2019_2,1001009,85,222,234,161,3,89,6,12,96.0,1463.0,1596.0,325.0,48.0,434.0,44.0,133.0


In [42]:
#### 상권/배후지 추정매출 (11, 19)

In [43]:
pred_sel_df = multi_columns_merging_svc_cd('상권-추정매출', '상권배후지-추정매출')
pred_sel_df

,기준_년_분기_코드,상권_코드,서비스_업종_코드_x,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,...,hl_시간대_건수~21_매출_건수,hl_시간대_건수~24_매출_건수,hl_남성_매출_건수,hl_여성_매출_건수,hl_연령대_10_매출_건수,hl_연령대_20_매출_건수,hl_연령대_30_매출_건수,hl_연령대_40_매출_건수,hl_연령대_50_매출_건수,hl_연령대_60_이상_매출_건수
0,2019_1,1000001,23,174,4628412627,298119,1526,777,274,276,...,324357.0,59957.0,641407.0,614200.0,27515.0,344152.0,367984.0,259400.0,172561.0,83995.0
1,2019_1,1000002,26,132,8949764495,343625,1954,647,469,385,...,697191.0,300242.0,1640615.0,1171936.0,35453.0,698576.0,748947.0,571552.0,448161.0,309875.0
2,2019_1,1000003,33,208,9423597217,358803,2668,632,531,557,...,1186520.0,391994.0,2242472.0,1961696.0,45516.0,1310311.0,1019162.0,799525.0,634699.0,394932.0
3,2019_1,1000004,24,99,3400775877,378584,1825,576,458,323,...,214459.0,87125.0,483463.0,338517.0,30595.0,445278.0,87297.0,85098.0,122891.0,50811.0
4,2019_1,1000005,17,55,2510984181,89060,1081,621,129,137,...,98776.0,54787.0,171798.0,125619.0,2377.0,85788.0,77513.0,55186.0,49603.0,26950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2019_2,1000624,22,71,1203742917,74987,1689,515,296,423,...,259085.0,120599.0,481503.0,385928.0,17890.0,139059.0,165732.0,208547.0,184964.0,151236.0
1634,2019_2,1000625,21,108,7037150341,359324,1481,621,281,280,...,703926.0,312485.0,1208029.0,1054515.0,24494.0,579730.0,732977.0,452411.0,297555.0,175362.0
1635,2019_2,1000626,27,80,4315840419,80982,1939,764,299,429,...,465188.0,116023.0,1181231.0,889820.0,6986.0,420300.0,569409.0,473151.0,336111.0,265087.0
1636,2019_2,1000627,31,151,2700876157,168910,2081,1021,550,447,...,833909.0,244648.0,1552983.0,1465445.0,28462.0,434331.0,655527.0,725325.0,647163.0,527608.0


In [44]:
pred_sel_df.columns.to_list()
yo_columns = pred_sel_df.filter(like='요일_').columns
si_columns = pred_sel_df.filter(like='시간대_').columns
age_columns = pred_sel_df.filter(like='연령대_').columns
# yo_columns = list(map( lambda s: s.replace('\n',''), yo_columns ))
# pred_sel_df.drop(yo_columns.to_list(), axis=1, inplace=True)
# pred_sel_df.drop(si_columns.to_list(), axis=1, inplace=True)
# pred_sel_df.drop(age_columns.to_list(), axis=1, inplace=True)
pred_sel_df.drop(yo_columns, axis=1, inplace=True)
pred_sel_df.drop(si_columns, axis=1, inplace=True)
pred_sel_df.drop(age_columns, axis=1, inplace=True)

# yo_columns.to_list()

In [45]:
pred_sel_df.columns.to_list()
pred_sel_df.rename(columns={'서비스_업종_코드_x':'서비스_업종_코드', '서비스_업종_코드_y':'hl_서비스_업종_코드'}, inplace=True)
pred_sel_df[['주중_매출_금액','주말_매출_금액']]

,주중_매출_금액,주말_매출_금액
0,3180426031,1447986596
1,6367720225,2582044270
2,7245208634,2178388583
3,2554418559,846357318
4,1505935110,1005049071
...,...,...
1633,850945539,352797378
1634,4731102004,2306048337
1635,3402354976,913485443
1636,1812356236,888519921


In [46]:
pred_sel_df['주중_매출_비율'] = pred_sel_df['주중_매출_금액'] / (pred_sel_df['주중_매출_금액'] + pred_sel_df['주말_매출_금액'])*100
pred_sel_df['주말_매출_비율'] = pred_sel_df['주말_매출_금액'] / (pred_sel_df['주중_매출_금액'] + pred_sel_df['주말_매출_금액'])*100
pred_sel_df['남성_매출_비율'] = pred_sel_df['남성_매출_금액'] / (pred_sel_df['남성_매출_금액'] + pred_sel_df['여성_매출_금액'])*100
pred_sel_df['여성_매출_비율'] = pred_sel_df['여성_매출_금액'] / (pred_sel_df['남성_매출_금액'] + pred_sel_df['여성_매출_금액'])*100

pred_sel_df['hl_주중_매출_비율'] = pred_sel_df['hl_주중_매출_금액'] / (pred_sel_df['hl_주중_매출_금액'] + pred_sel_df['hl_주말_매출_금액'])*100
pred_sel_df['hl_주말_매출_비율'] = pred_sel_df['hl_주말_매출_금액'] / (pred_sel_df['hl_주중_매출_금액'] + pred_sel_df['hl_주말_매출_금액'])*100
pred_sel_df['hl_남성_매출_비율'] = pred_sel_df['hl_남성_매출_금액'] / (pred_sel_df['hl_남성_매출_금액'] + pred_sel_df['hl_여성_매출_금액'])*100
pred_sel_df['hl_여성_매출_비율'] = pred_sel_df['hl_여성_매출_금액'] / (pred_sel_df['hl_남성_매출_금액'] + pred_sel_df['hl_여성_매출_금액'])*100
pred_sel_df[['주중_매출_비율','주말_매출_비율', 'hl_주중_매출_비율','hl_주말_매출_비율']]

,주중_매출_비율,주말_매출_비율,hl_주중_매출_비율,hl_주말_매출_비율
0,68.715266,31.284734,73.147848,26.852152
1,71.149584,28.850416,73.088294,26.911706
2,76.883683,23.116317,72.431740,27.568260
3,75.112817,24.887183,74.960975,25.039025
4,59.973899,40.026101,60.906421,39.093579
...,...,...,...,...
1633,70.691634,29.308366,75.142628,24.857372
1634,67.230367,32.769633,67.326315,32.673685
1635,78.834124,21.165876,80.801657,19.198343
1636,67.102530,32.897470,75.521138,24.478862


In [47]:
pred_sel_df

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,...,hl_남성_매출_비율,hl_여성_매출_비율,hl_주중_매출_금액,hl_주말_매출_금액,hl_남성_매출_금액,hl_여성_매출_금액,hl_주중_매출_건수,hl_주말_매출_건수,hl_남성_매출_건수,hl_여성_매출_건수
0,2019_1,1000001,23,174,4628412627,298119,68.715266,31.284734,40.179135,59.820865,...,46.790656,53.209344,1.698039e+10,6.233403e+09,9.444083e+09,1.073961e+10,980879.0,334369.0,641407.0,614200.0
1,2019_1,1000002,26,132,8949764495,343625,71.149584,28.850416,41.353139,58.646861,...,56.115140,43.884860,3.323853e+10,1.223870e+10,2.283884e+10,1.786112e+10,2136886.0,773772.0,1640615.0,1171936.0
2,2019_1,1000003,33,208,9423597217,358803,76.883683,23.116317,50.488938,49.511062,...,52.148198,47.851802,6.862770e+10,2.612040e+10,4.402554e+10,4.039836e+10,3250855.0,1157636.0,2242472.0,1961696.0
3,2019_1,1000004,24,99,3400775877,378584,75.112817,24.887183,58.577910,41.422090,...,57.065415,42.934585,4.952166e+09,1.654159e+09,3.726967e+09,2.804076e+09,650493.0,172942.0,483463.0,338517.0
4,2019_1,1000005,17,55,2510984181,89060,59.973899,40.026101,46.854058,53.145942,...,46.837424,53.162576,3.135343e+09,2.012461e+09,2.191419e+09,2.487358e+09,169050.0,138295.0,171798.0,125619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2019_2,1000624,22,71,1203742917,74987,70.691634,29.308366,49.818500,50.181500,...,53.544702,46.455298,1.392247e+10,4.605588e+09,9.274562e+09,8.046595e+09,641582.0,234953.0,481503.0,385928.0
1634,2019_2,1000625,21,108,7037150341,359324,67.230367,32.769633,67.899134,32.100866,...,56.149134,43.850866,3.586015e+10,1.740305e+10,2.787033e+10,2.176593e+10,1647924.0,669297.0,1208029.0,1054515.0
1635,2019_2,1000626,27,80,4315840419,80982,78.834124,21.165876,42.166589,57.833411,...,53.221974,46.778026,3.137855e+10,7.455492e+09,1.855149e+10,1.630533e+10,1825542.0,330052.0,1181231.0,889820.0
1636,2019_2,1000627,31,151,2700876157,168910,67.102530,32.897470,60.917854,39.082146,...,50.673676,49.326324,4.232204e+10,1.371795e+10,2.706800e+10,2.634830e+10,2261188.0,800906.0,1552983.0,1465445.0


In [48]:
#### 상권/배후지 상주인구 (06, 05)

In [49]:
live_pop_df = multi_columns_merging('상권-상주인구', '상권배후지-상주인구')
delete_columns = live_pop_df.filter(like='연령대_').columns
live_pop_df.drop(delete_columns, axis=1, inplace=True)

In [50]:
live_pop_df

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,hl_비_아파트_가구_수
366,2019_2,1000979,3269,1651,1618,1656,122,1534,22129.0,11025.0,11104.0,9670.0,2935.0,6735.0
367,2019_2,1000978,3651,1791,1860,1599,333,1266,21533.0,10838.0,10695.0,10181.0,1087.0,9094.0
368,2019_2,1000977,2767,1379,1388,1274,114,1160,23136.0,11442.0,11694.0,9377.0,3085.0,6292.0
369,2019_2,1000976,2914,1498,1416,1274,0,1274,13231.0,6793.0,6438.0,4666.0,767.0,3899.0
370,2019_2,1000975,1694,915,779,821,0,821,4617.0,2456.0,2161.0,2112.0,0.0,2112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,2019_1,1000005,1694,826,868,725,0,725,6102.0,2958.0,3144.0,2607.0,8.0,2599.0
2958,2019_1,1000004,1453,711,742,884,0,884,9137.0,4521.0,4616.0,4910.0,257.0,4653.0
2959,2019_1,1000003,1214,682,532,805,0,805,3512.0,2033.0,1479.0,2394.0,150.0,2244.0
2960,2019_1,1000002,841,383,458,775,8,767,11047.0,5624.0,5423.0,6257.0,2285.0,3972.0


In [51]:
#### 상권/배후지 직장인구 (12, 14)

In [52]:
job_pop_df = multi_columns_merging('상권-직장인구', '상권배후지-직장인구')
job_pop_df
delete_columns = job_pop_df.filter(like='연령대_').columns
job_pop_df.drop(delete_columns, axis=1, inplace=True)

In [53]:
job_pop_df

,기준_년_분기_코드,상권_코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수
417,2019_2,1000918,204,93,111,14517.0,7537.0,6980.0
418,2019_2,1000917,837,420,417,15288.0,9326.0,5962.0
419,2019_2,1000916,1383,854,529,26050.0,14721.0,11329.0
420,2019_2,1000915,477,278,199,11412.0,7492.0,3920.0
421,2019_2,1000914,4389,2637,1752,26569.0,16745.0,9824.0
...,...,...,...,...,...,...,...,...
2980,2019_1,1000005,1080,692,388,1820.0,994.0,826.0
2981,2019_1,1000004,104,58,46,2196.0,1194.0,1002.0
2982,2019_1,1000003,4822,2188,2634,44890.0,32318.0,12572.0
2983,2019_1,1000002,486,254,232,13270.0,6153.0,7117.0


In [54]:
#### 상권/배후지 추정유동인구 (01, 03)

In [55]:
pred_pop_df = load_to_df('상권-추정유동인구')
pred_pop_df = delete_age_time(pred_pop_df, flag='')
pred_pop_df
hl_pred_pop_df = load_to_df('상권배후지-추정유동인구')
hl_pred_pop_df = delete_age_time(hl_pred_pop_df, flag='hl_')
hl_pred_pop_df

,기준_년_분기_코드,상권_코드,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_주말_유동인구수,hl_주중_유동인구수
0,2019_1,1000645,161694,87243,74446,50715,110987
1,2019_1,1000646,190850,108281,82567,67141,123714
2,2019_1,1000647,238761,118531,120230,81719,157045
3,2019_1,1000659,1461213,844363,616850,394595,1066622
4,2019_1,1000660,1871791,1006595,865199,485040,1386761
...,...,...,...,...,...,...,...
2015,2019_2,1000884,1462461,717503,744954,432852,1029604
2016,2019_2,1000885,2026927,1069686,957239,445283,1581647
2017,2019_2,1000931,1401131,711644,689491,319912,1081222
2018,2019_2,1000932,2133743,1164525,969217,594037,1539700


In [56]:
all_df = pd.merge(left=pred_pop_df, right=hl_pred_pop_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
del pred_pop_df, hl_pred_pop_df
    
# jumpo_df.index.name
# 멀티 인덱스 제거
# all_df.dropna(axis=0).reset_index(['상권_코드','기준_년_분기_코드'])
all_pred_pop_df = all_df.dropna(axis=0)
all_pred_pop_df

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,주말_유동인구수,주중_유동인구수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_주말_유동인구수,hl_주중_유동인구수
0,2019_1,1000001,118266,61268,56997,31451,86815,471903.0,253713.0,218188.0,112607.0,359301.0
1,2019_1,1000002,246619,147358,99261,90234,156385,2186033.0,1237827.0,948202.0,686921.0,1499105.0
2,2019_1,1000003,144480,80073,64407,36815,107665,1818586.0,1015350.0,803234.0,477893.0,1340682.0
3,2019_1,1000012,166390,70339,96051,45018,121377,796819.0,357225.0,439599.0,212116.0,584703.0
4,2019_1,1000013,65518,33437,32080,18769,46747,1172123.0,649329.0,522791.0,330935.0,841181.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2782,2019_2,1000917,177560,85271,92289,44432,133129,1482344.0,722350.0,760001.0,332536.0,1149815.0
2783,2019_2,1000918,88300,38028,50274,22316,65983,1745604.0,768868.0,976735.0,437412.0,1308189.0
2784,2019_2,1000964,325820,170203,155619,79801,246018,1407370.0,727295.0,680081.0,357658.0,1049710.0
2785,2019_2,1000965,252095,124397,127697,82277,169820,1614498.0,800189.0,814313.0,482523.0,1131979.0


In [57]:
hl_income_expn = load_to_df('상권배후지-소득소비')

In [58]:
# 1009 개 이지만 상권/배후지 통합데이터 고려시 1008 개.

# # pop
#### live_pop_df
#### job_pop_df
#### all_pred_pop_df

# # place
#### apart_df
#### jip_df
#### jumpo_df

# # selling
#### pred_sel


In [59]:
# sheet_list = [ '상권배후지-추정유동인구', '상권배후지-집객시설', '상권배후지-상주인구', 
# '상권배후지-아파트', '상권배후지-점포','상권-상권변화지표',
# '상권배후지-직장인구', '자치구별 상권변화지표', '상권배후지-추정매출' ]

# sheet_list = [ 'live_pop_df', 'job_pop_df', 'all_pred_pop_df', 'apart_df',
#               'jip_df', 'jumpo_df', 'pred_sel' ]


In [60]:
dfs = [ live_pop_df, job_pop_df,
#        pred_sel_df,
       apart_df,
        jip_df, jumpo_df, all_pred_pop_df]

def multi_frame_merge(dfs):
    
    for idx, df in enumerate(dfs):
        if idx == 0:
            tmp_df = df
        else:
            tmp_df = pd.merge(left=tmp_df, right=df, how='left', \
                              on=['기준_년_분기_코드','상권_코드'], sort=False)
    return tmp_df

all_dfs = multi_frame_merge(dfs)
# all_dfs.columns.to_list()

In [61]:
all_dfs[all_dfs['기준_년_분기_코드']=='2019_2'].head(10)

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,...,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,주말_유동인구수,주중_유동인구수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_주말_유동인구수,hl_주중_유동인구수
0,2019_2,1000979,3269,1651,1618,1656,122,1534,22129.0,11025.0,...,310182,158608,151577,88991,221195,2039133.0,1038158.0,1000980.0,598717.0,1440422.0
1,2019_2,1000978,3651,1791,1860,1599,333,1266,21533.0,10838.0,...,370323,194391,175933,110688,259634,2312395.0,1173915.0,1138491.0,682491.0,1629908.0
2,2019_2,1000977,2767,1379,1388,1274,114,1160,23136.0,11442.0,...,290862,150339,140519,80533,210329,2137446.0,1070557.0,1066890.0,619898.0,1517545.0
3,2019_2,1000976,2914,1498,1416,1274,0,1274,13231.0,6793.0,...,76753,39129,37626,22016,54735,241267.0,121259.0,120011.0,66897.0,174378.0
4,2019_2,1000975,1694,915,779,821,0,821,4617.0,2456.0,...,67380,42743,24636,16465,50910,121743.0,80199.0,41539.0,29492.0,92248.0
5,2019_2,1000974,2695,1318,1377,1071,0,1071,12517.0,6218.0,...,102090,48003,54089,25156,76935,387338.0,171713.0,215624.0,137270.0,250066.0
6,2019_2,1000973,4286,2169,2117,2174,107,2067,31916.0,15901.0,...,281149,138620,142531,84192,196957,4520602.0,2327998.0,2192615.0,1340639.0,3179983.0
7,2019_2,1000972,5439,2715,2724,2458,334,2124,28177.0,14075.0,...,324159,162089,162073,98867,225297,4111868.0,2134700.0,1977170.0,1217592.0,2894279.0
8,2019_2,1000971,2602,1347,1255,1214,0,1214,26672.0,13585.0,...,176873,89717,87157,55765,121110,2130460.0,1084720.0,1045731.0,670406.0,1460074.0
9,2019_2,1000970,1602,821,781,623,0,623,17436.0,8854.0,...,29454,13885,15569,8163,21292,242093.0,123067.0,119030.0,65439.0,176664.0


In [62]:
all_dfs[all_dfs['기준_년_분기_코드']=='2019_2'].head(10)

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,...,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,주말_유동인구수,주중_유동인구수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_주말_유동인구수,hl_주중_유동인구수
0,2019_2,1000979,3269,1651,1618,1656,122,1534,22129.0,11025.0,...,310182,158608,151577,88991,221195,2039133.0,1038158.0,1000980.0,598717.0,1440422.0
1,2019_2,1000978,3651,1791,1860,1599,333,1266,21533.0,10838.0,...,370323,194391,175933,110688,259634,2312395.0,1173915.0,1138491.0,682491.0,1629908.0
2,2019_2,1000977,2767,1379,1388,1274,114,1160,23136.0,11442.0,...,290862,150339,140519,80533,210329,2137446.0,1070557.0,1066890.0,619898.0,1517545.0
3,2019_2,1000976,2914,1498,1416,1274,0,1274,13231.0,6793.0,...,76753,39129,37626,22016,54735,241267.0,121259.0,120011.0,66897.0,174378.0
4,2019_2,1000975,1694,915,779,821,0,821,4617.0,2456.0,...,67380,42743,24636,16465,50910,121743.0,80199.0,41539.0,29492.0,92248.0
5,2019_2,1000974,2695,1318,1377,1071,0,1071,12517.0,6218.0,...,102090,48003,54089,25156,76935,387338.0,171713.0,215624.0,137270.0,250066.0
6,2019_2,1000973,4286,2169,2117,2174,107,2067,31916.0,15901.0,...,281149,138620,142531,84192,196957,4520602.0,2327998.0,2192615.0,1340639.0,3179983.0
7,2019_2,1000972,5439,2715,2724,2458,334,2124,28177.0,14075.0,...,324159,162089,162073,98867,225297,4111868.0,2134700.0,1977170.0,1217592.0,2894279.0
8,2019_2,1000971,2602,1347,1255,1214,0,1214,26672.0,13585.0,...,176873,89717,87157,55765,121110,2130460.0,1084720.0,1045731.0,670406.0,1460074.0
9,2019_2,1000970,1602,821,781,623,0,623,17436.0,8854.0,...,29454,13885,15569,8163,21292,242093.0,123067.0,119030.0,65439.0,176664.0


In [63]:
pred_sel_df.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '점포수', '당월_매출_금액', '당월_매출_건수',
       '주중_매출_비율', '주말_매출_비율', '남성_매출_비율', '여성_매출_비율', '주중_매출_금액', '주말_매출_금액',
       '남성_매출_금액', '여성_매출_금액', '주중_매출_건수', '주말_매출_건수', '남성_매출_건수', '여성_매출_건수',
       'hl_서비스_업종_코드', 'hl_점포수', 'hl_당월_매출_금액', 'hl_당월_매출_건수', 'hl_주중_매출_비율',
       'hl_주말_매출_비율', 'hl_남성_매출_비율', 'hl_여성_매출_비율', 'hl_주중_매출_금액',
       'hl_주말_매출_금액', 'hl_남성_매출_금액', 'hl_여성_매출_금액', 'hl_주중_매출_건수',
       'hl_주말_매출_건수', 'hl_남성_매출_건수', 'hl_여성_매출_건수'],
      dtype='object')

In [64]:
jumpo_df.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '점포_수', '유사_업종_점포_수', '개업_율',
       '개업_점포_수', '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수', 'hl_서비스_업종_코드', 'hl_점포_수',
       'hl_유사_업종_점포_수', 'hl_개업_율', 'hl_개업_점포_수', 'hl_폐업_률', 'hl_폐업_점포_수',
       'hl_프랜차이즈_점포_수'],
      dtype='object')

In [65]:
pred_sel_df[['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '점포수']]

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포수
0,2019_1,1000001,23,174
1,2019_1,1000002,26,132
2,2019_1,1000003,33,208
3,2019_1,1000004,24,99
4,2019_1,1000005,17,55
...,...,...,...,...
1633,2019_2,1000624,22,71
1634,2019_2,1000625,21,108
1635,2019_2,1000626,27,80
1636,2019_2,1000627,31,151


In [66]:
pred_sel_df

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포수,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,남성_매출_비율,여성_매출_비율,...,hl_남성_매출_비율,hl_여성_매출_비율,hl_주중_매출_금액,hl_주말_매출_금액,hl_남성_매출_금액,hl_여성_매출_금액,hl_주중_매출_건수,hl_주말_매출_건수,hl_남성_매출_건수,hl_여성_매출_건수
0,2019_1,1000001,23,174,4628412627,298119,68.715266,31.284734,40.179135,59.820865,...,46.790656,53.209344,1.698039e+10,6.233403e+09,9.444083e+09,1.073961e+10,980879.0,334369.0,641407.0,614200.0
1,2019_1,1000002,26,132,8949764495,343625,71.149584,28.850416,41.353139,58.646861,...,56.115140,43.884860,3.323853e+10,1.223870e+10,2.283884e+10,1.786112e+10,2136886.0,773772.0,1640615.0,1171936.0
2,2019_1,1000003,33,208,9423597217,358803,76.883683,23.116317,50.488938,49.511062,...,52.148198,47.851802,6.862770e+10,2.612040e+10,4.402554e+10,4.039836e+10,3250855.0,1157636.0,2242472.0,1961696.0
3,2019_1,1000004,24,99,3400775877,378584,75.112817,24.887183,58.577910,41.422090,...,57.065415,42.934585,4.952166e+09,1.654159e+09,3.726967e+09,2.804076e+09,650493.0,172942.0,483463.0,338517.0
4,2019_1,1000005,17,55,2510984181,89060,59.973899,40.026101,46.854058,53.145942,...,46.837424,53.162576,3.135343e+09,2.012461e+09,2.191419e+09,2.487358e+09,169050.0,138295.0,171798.0,125619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2019_2,1000624,22,71,1203742917,74987,70.691634,29.308366,49.818500,50.181500,...,53.544702,46.455298,1.392247e+10,4.605588e+09,9.274562e+09,8.046595e+09,641582.0,234953.0,481503.0,385928.0
1634,2019_2,1000625,21,108,7037150341,359324,67.230367,32.769633,67.899134,32.100866,...,56.149134,43.850866,3.586015e+10,1.740305e+10,2.787033e+10,2.176593e+10,1647924.0,669297.0,1208029.0,1054515.0
1635,2019_2,1000626,27,80,4315840419,80982,78.834124,21.165876,42.166589,57.833411,...,53.221974,46.778026,3.137855e+10,7.455492e+09,1.855149e+10,1.630533e+10,1825542.0,330052.0,1181231.0,889820.0
1636,2019_2,1000627,31,151,2700876157,168910,67.102530,32.897470,60.917854,39.082146,...,50.673676,49.326324,4.232204e+10,1.371795e+10,2.706800e+10,2.634830e+10,2261188.0,800906.0,1552983.0,1465445.0


In [67]:
jumpo_df[['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '점포_수']]

,기준_년_분기_코드,상권_코드,서비스_업종_코드,점포_수
0,2019_1,1000001,71,233
1,2019_1,1000002,80,142
2,2019_1,1000003,81,322
3,2019_1,1000004,56,97
4,2019_1,1000005,58,77
...,...,...,...,...
2015,2019_2,1001006,77,144
2016,2019_2,1001007,78,169
2017,2019_2,1001008,80,206
2018,2019_2,1001009,85,222


In [68]:
all_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 0 to 2019
Data columns (total 92 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   기준_년_분기_코드     2020 non-null   object 
 1   상권_코드          2020 non-null   object 
 2   총_상주인구_수       2020 non-null   int64  
 3   남성_상주인구_수      2020 non-null   int64  
 4   여성_상주인구_수      2020 non-null   int64  
 5   총_가구_수         2020 non-null   int64  
 6   아파트_가구_수       2020 non-null   int64  
 7   비_아파트_가구_수     2020 non-null   int64  
 8   hl_총_상주인구_수    2020 non-null   float64
 9   hl_남성_상주인구_수   2020 non-null   float64
 10  hl_여성_상주인구_수   2020 non-null   float64
 11  hl_총_가구_수      2020 non-null   float64
 12  hl_아파트_가구_수    2020 non-null   float64
 13  hl_비_아파트_가구_수  2020 non-null   float64
 14  총_직장_인구_수      2020 non-null   int64  
 15  남성_직장_인구_수     2020 non-null   int64  
 16  여성_직장_인구_수     2020 non-null   int64  
 17  hl_총_직장_인구_수   2020 non-null   float64
 18  hl_남성_직장